In [8]:
import os
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [9]:
def process_description(soup):
    description = soup.find(attrs={'class':'description'})
    
    aggregated_text = {}
    aggregated_text["university_name"] = soup.find('h2').find('a').text
    aggregated_text['FullText'] = description.text
    headers = re.findall(r"<dt>[^<]*</dt>", str(description))
    texts = re.findall(r"<dd>[^<]*</dd>", str(description))
    if (len(headers) == 0) or (len(headers) != len(texts)):
         return aggregated_text
    for header, text in zip(headers, texts):
        header = header[4:-5]
        text = text[4:-5]
        aggregated_text[header] = text
    return aggregated_text

In [10]:
time.sleep(5)

In [12]:
#allPhd = {}
outJson = "scraped_result.json"
page = 1
base = "https://scholarshipdb.net/PhD-scholarships-in-United-States?page={}"
while True:
    html_text = requests.get(base.format(page), verify=False)
    soup = BeautifulSoup(html_text.content, "html.parser")
    all_links = [link.get('href') for link in soup.find_all("a")]
    phd_links = [link for link in all_links if link.endswith('.html')]
    if len(phd_links) == 0:
        break
    print(f'Processing page {page} - found {len(phd_links)} links.')
    for link in phd_links:
        time.sleep(1)
        url = 'https://scholarshipdb.net' + link
        html_text_phd = requests.get(url, verify=False)
        soup_phd = BeautifulSoup(html_text_phd.content, "html.parser")
        allPhd[url] = process_description(soup_phd)
    with open(outJson, "w") as fp:
        json.dump(allPhd, fp)
    time.sleep(5)
    page += 1

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scholarshipdb.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Processing page 1 - found 10 links.


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scholarshipdb.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


TypeError: list indices must be integers or slices, not str

In [139]:
with open(f"scraped_result-backup-page{page}.json", "w") as fp:
    json.dump(allPhd, fp)

In [148]:
Scrped_US_ScholrshipDB = pd.DataFrame(allPhd).T
Scrped_US_ScholrshipDB.to_csv('../Data/Scrped_US_ScholrshipDB.csv', index=True)

In [118]:
for phd_url, descriptions in allPhd.items():
    print('https://scholarshipdb.net' + phd_url)
    print(descriptions.keys())

https://scholarshipdb.net/scholarships-in-United-States/Ph-D-Project-Mechanobiology-Of-Wound-Regeneration-University-Of-Florida=7iAtU7Zj7RGUYQAlkGUTnw.html
dict_keys(['FullText'])
https://scholarshipdb.net/scholarships-in-United-States/Ph-D-Position-Available-On-Climate-Change-And-Ecosystem-Services-University-Of-Florida-University-Of-Florida=iJ1EJDRn7RGUYQAlkGUTnw.html
dict_keys(['FullText', 'Agency', 'Location', 'Job Category', 'Salary', 'Start Date', 'Last Date to Apply', 'Website', 'Description', 'Qualifications', 'Contact Person', 'Contact eMail'])
https://scholarshipdb.net/scholarships-in-United-States/Ph-D-Student-Opportunities-In-Atmospheric-Chemistry-At-The-University-Of-Rochester-University-Of-Rochester=N9WPqvlk7RGUYQAlkGUTnw.html
dict_keys(['FullText'])
https://scholarshipdb.net/scholarships-in-United-States/Ph-D-And-Ms-Assistantships-In-Forestry-Forest-Science-Ecology-School-Of-Forest-Resources-And-Environmental-Science=GRZy8-pk7RGUYQAlkGUTnw.html
dict_keys(['FullText'])
ht